# Kalman Filter Implementation
Unit implementation of the KF.
### Input
* From camera: $z^t_{camera} = (p1;p2) = ((p^t_{1x},p^t_{1y});(p^t_{2x},p^t_{2y}))$ expressed in $[mm]$. We choose p1 being the center of rotation of the robot and p2 another point drawn on the robot behind this point. 
* From wheel: $x^t_{wheel}=(d,alpha)$ expressed in $[mm]$ and $[deg]$.

### Predict
$\hat{x}_{k} = F_k x_{k-1} + B_{k} u_k$

$P_{k} = F_k P_{k-1} F_k^T + Q_k$

With:

* $\hat{x}_{k} = \begin{pmatrix} \hat{x}_{k} \\ \hat{y}_{k} \\ \hat{\alpha}_{k} \end{pmatrix}$;  $u_{k} = \begin{pmatrix} d_{\Delta t} \\ \alpha_{\Delta t} \end{pmatrix}$
* $F_k = \begin{pmatrix} 1 \ 0 \ 0 \\ 0 \ 1 \ 0 \\ 0 \ 0 \ 1 \end{pmatrix} $;  $B_k = \begin{pmatrix} \sin(\alpha) \ 0 \\ \cos(\alpha) \ 0 \\ 0 \ 1 \end{pmatrix} $
* $Q_k$ being the covariance of the measurement of wheels, TBD (to be determined)

### Update

* $y_k = z_k - H_k \hat{x}_k$
* $S_k = H_k P_k H_k^T + R_k$
* $K_k = P_k H_k^T S_k^{-1}$
* $\hat{x}_k = \hat{x}_k + K_k \tilde{y}_k$
* $P_k = (I - K_k H_k) P_k$

With:
* H_k
* z_k camera measurement

In [31]:
import numpy as np
import math

# STATE SPACE VARIABLE INITIALIZATION
x = np.array([[0], [0], [45]])                                              # Initial state  (x = (x,y,alpha))
F = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])                             # Define the state transition matrix (F)
u = np.array([[0], [0]])                                                    # Initial control input  (u = (Deltad, Deltaalpha))
B = np.array([[math.sin(math.radians(x[2])), 0], [math.cos(math.radians(x[2])), 0], [0, 1]])            # Define the control matrix (B)

# KALMAN FILETER VARIABLE INITIALIZATION
y = np.array([[0], [0], [0]])                                               # Initial output (y = (x,y,alpha))
P = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])                             # Initial prediction (P)
H = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])                             # Define the measurement matrix (H)

# TO BE DEFINE
Q = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])                             # Define the process noise covariance (Q)
R = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])                             # Define the measurement noise covariance (R)


def predict(x, u, P):
    x_hat = F.dot(x) + B.dot(u)
    P = F.dot(P).dot(F.T) + Q
    return x_hat, P

def update(x_hat_pred, P_pred, raw_camera_measurement):
    z = get_camera_measurement(raw_camera_measurement)

    y = z - H.dot(x_hat_pred)
    S = H.dot(P_pred).dot(H.T) + R
    K = P.dot(H.T).dot(np.linalg.inv(S))
    x_hat_upd = x_hat_pred + K.dot(y)
    P_upd = P_pred - K.dot(H).dot(P_pred)

    return x_hat_upd, P_upd

def get_camera_measurement(raw_camera_measurement):                         # raw_camera_measurement = [p1;p2] = [p1_x, p1_y, p2_x, p2_y]
    p1_x = raw_camera_measurement[0][0]
    p1_y = raw_camera_measurement[0][1]
    p2_x = raw_camera_measurement[1][0]
    p2_y = raw_camera_measurement[1][1]

    d_Detlat = math.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)
    alpha_Deltat = math.atan2(p2_y - p1_y, p2_x - p1_x) - x[2]
    x_Deltat = x[0] + d_Detlat * math.cos(math.radians(x[2] + alpha_Deltat))
    y_Deltat = x[1] + d_Detlat * math.sin(math.radians(x[2] + alpha_Deltat))

    z = np.array([[x_Deltat], [y_Deltat], [alpha_Deltat]])
    return z

# Testing code

In [32]:
# Input
raw_camera_measurement = np.array([[0,0],[1,1]])                            # From Camera ((p1_x, p1_y); (p2_x, p2_y))
raw_wheel_measurement = np.array([[math.sqrt(2)],[0]])                                 # From Wheel Encoder (d_Deltat; alpha_Deltat)

x_hat_pred, P_pred = predict(x, raw_wheel_measurement, P)
x_hat, P = update(x_hat_pred, P_pred, raw_camera_measurement)

print("Updated state estimate:\n", x_hat_pred)
print("Updated error covariance:\n", P)

Updated state estimate:
 [[ 1.]
 [ 1.]
 [45.]]
Updated error covariance:
 [[1.33333333 0.         0.        ]
 [0.         1.33333333 0.        ]
 [0.         0.         1.33333333]]
